1. Choose runtime type as T4 GPU

In [1]:
import torch

if torch.cuda.is_available():
    print("CUDA is available.")
    print("CUDA version:", torch.version.cuda)
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available.")


CUDA is available.
CUDA version: 12.1
GPU: Tesla T4


2. Upload the model

In [ ]:
%cd /content
!git clone --recursive https://github.com/vaishvijs13/se101-model
!pip install -q plyfile

%cd /content/gaussian-splatting
!pip install -q /content/gaussian-splatting/submodules/diff-gaussian-rasterization
!pip install -q /content/gaussian-splatting/submodules/simple-knn

3. Add folder of images from Drive (change folder name to 'input' and move this subfolder to a larger folder, ie. 'colmap')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

4. COLMAP dependencies:

In [ ]:
!sudo apt-get install -y \
    git \
    cmake \
    ninja-build \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libboost-regex-dev \
    libboost-test-dev \
    libeigen3-dev \
    libflann-dev \
    libfreeimage-dev \
    libmetis-dev \
    libgoogle-glog-dev \
    libgtest-dev \
    libsqlite3-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libceres-dev \
    libsuitesparse-dev \
    libgflags-dev \
    libpng-dev \
    libjpeg-dev \
    libtiff-dev \
    libxxf86vm1 \
    libxxf86vm-dev \
    libxi-dev \
    libxrandr-dev

5. Build COLMAP

In [5]:
%cd /content
!git clone https://github.com/colmap/colmap.git
%cd colmap
!mkdir build
%cd build
!cmake ..
!make -j$(nproc)
!sudo make install

/content
Cloning into 'colmap'...
remote: Enumerating objects: 23928, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 23928 (delta 100), reused 47 (delta 44), pack-reused 23764 (from 3)
Receiving objects: 100% (23928/23928), 69.70 MiB | 21.33 MiB/s, done.
Resolving deltas: 100% (18435/18435), done.
/content/colmap
/content/colmap/build
-- Enabling LSD support
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found FreeImage
--   Includes : /usr/include
--   Libraries

6. Convert the dataset using COLMAP <br />

In [ ]:
!python colmap.py -s /content/gaussian-splatting/colmap

7. Train the converted dataset

In [ ]:
%cd /content/gaussian-splatting
!python train.py -s /content/gaussian-splatting/colmap --iterations 30000 --save_iterations 10000 20000 30000

2024-12-16 22:12:29.704416: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-16 22:12:29.723956: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-16 22:12:29.729797: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-16 22:12:29.745235: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-16 22:12:31.143532: W tensorflow/comp

In [ ]:
file_path = '/content/gaussian-splatting/output/72781dcf-c/point_cloud/iteration_20000/point_cloud.ply'  # replace with the given path
drive_path = '/content/drive/My Drive/drone.ply'
!cp "{file_path}" "{drive_path}"
print(f"File saved to Google Drive: {drive_path}")